In [ ]:
# 导入包(PypI)
import re
import time
import random
import os
import requests
import pymongo
import pandas as pd

from datetime import datetime
from tqdm import tqdm

from scrapy.selector import Selector
from scrapy.http import TextResponse

from seleniumwire import webdriver  # 使用 seleniumwire 的 webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# 创建 'images' 文件夹，如果不存在
images_folder = 'images'
os.makedirs(images_folder, exist_ok=True)

# 数据清洗可能需要使用的方法
def extract_number(List):
    list_cleaned = []
    for x in List:
        if x is None or not isinstance(x, str):
            list_cleaned.append(None)
            continue

        # Find all numeric patterns in the string
        numbers = re.findall(r'\d+\.?\d*', x)
        if numbers:
            # Join all found numbers (if multiple numbers are in one string)
            number = ''.join(numbers)
            list_cleaned.append(number)
        else:
            list_cleaned.append(None)

    return list_cleaned

def extract_large_number(List):
    list_cleaned = []
    for x in List:
        if x is None or not isinstance(x, str):
            list_cleaned.append(None)
            continue

        try:
            if '万' in x:
                number = x.replace('万', '')
                number = float(number) * 10000
                number = int(number)
            else:
                number = int(float(x))
            list_cleaned.append(number)
        except ValueError:
            # If conversion fails, append None or handle accordingly
            list_cleaned.append(None)

    return list_cleaned

def extract_date(List):
    list_cleaned = []
    for x in List:
        date_cleand = None  # Initialize with None
        if x is None or not isinstance(x, str):
            list_cleaned.append(date_cleand)
            continue

        match = re.search(r'(\d{4})-(\d{2})-(\d{2})', x)
        if match:
            date_cleand = match.group(0)
        else:
            match = re.search(r'(\d{2})-(\d{2})', x)
            if match:
                current_year = datetime.now().year
                month, day = match.groups()
                date_cleand = f'{current_year}-{month}-{day}'

        list_cleaned.append(date_cleand)

    return list_cleaned

# 配置Chrome浏览器
service = Service("/opt/homebrew/bin/chromedriver")  # 请确保 chromedriver 路径正确

options = Options()
options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')  # 远程调控模式启用
options.add_argument('--incognito')  # 隐身/无痕模式启用

# 初始化 Selenium Wire WebDriver
browser = webdriver.Chrome(service=service, options=options)
action = ActionChains(browser)

key_word = ""
num = 0

def check_login_status(browser):
    print("即将开始检查小红书登录状态...")
    print("爬取数据有账户封禁的风险，建议使用非主账号登录。")
    
    while True:
        page_source = browser.page_source
        if '登录探索更多内容' in page_source:
            print('暂未登录，请手动登录')
            print('检查时间:', time.ctime())
            time.sleep(10)
        else:
            print('登录成功')
            print('检查时间:', time.ctime())
            time.sleep(3)
            break

def check_page_load_status(browser, keyword):
    print("即将开始检查网页加载状态...")
    print("如果网页进入人机验证页面，请先手动完成验证。")
    
    while True:
        if keyword in browser.title:
            print('加载成功')
            print('检查时间:', time.ctime())
            break
        else:
            time.sleep(2)

def selenium_test():
    """
    登录状态检查，网页加载检查，根据用户输入进行搜索
    """
    global key_word, num
    browser.get('https://www.xiaohongshu.com/explore')
    
    check_login_status(browser)
    
    print("请在文本框中根据提示输入搜索关键词和笔记爬取数量。")
    keyword = input("搜索关键词：")
    try:
        num = int(input("笔记爬取数量："))
    except ValueError:
        print("请输入有效的整数作为爬取数量。")
        return
    
    url = f'https://www.xiaohongshu.com/search_result?keyword={keyword}&source=web_explore_feed'
    browser.get(url)
    time.sleep(3)

    check_page_load_status(browser, keyword)
    
selenium_test()

def change_mode(browser):
    # 更改模式为图文
    try:
        mode_button = browser.find_element(By.XPATH, '//*[@id="search-type"]/div/div/div[2]')
        mode_button.click()
        print('已自动更改模式为图文。')
    except Exception as e:
        print(f"更改模式失败: {e}")

selected_order_text = ''
def change_sort_order(browser, action):
    # 更改排序方式
    sort_order = {
        '综合': 1,
        '最新': 2,
        '最热': 3
    }
    print("请选择排序方式:")
    for order, idx in sort_order.items():
        print(f'{idx}. {order}')
    
    try:
        global selected_order_text
        selected_order_text = input("请输入排序方式对应的名称: ").strip()
        if selected_order_text not in sort_order:
            print("请输入有效的排序方式...")
            return
        
        selected_order_index = sort_order[selected_order_text]
    except Exception as e:
        print(f"处理排序选择时出错: {e}")
        return

    try:
        element = browser.find_element(By.XPATH, '//*[@id="global"]/div[2]/div[2]/div/div[1]/div[2]')
        action.move_to_element(element).perform()  # 模拟鼠标悬停
        menu = browser.find_element(By.CLASS_NAME, 'dropdown-items')
        option = menu.find_element(By.XPATH, f'/html/body/div[4]/div/li[{selected_order_index}]')
        option.click()  # 模拟鼠标点击

        print('已选择排序方式为:', selected_order_text)
        print('检查时间:', time.ctime())

    except Exception as e:
        print(f"更改排序方式失败: {e}")

change_mode(browser)
change_sort_order(browser, action)

def parsePage(html_content, authorName_list, likeNr_list, URL_list, userURL_list, num):
    """
    解析网页内容并更新数据列表。

    Args:
        html_content (str): 当前页面的HTML内容
        authorName_list (list): 存储作者名字的列表
        likeNr_list (list): 存储获赞数量的列表
        URL_list (list): 存储笔记URL的列表
        userURL_list (list): 存储用户URL的列表
        num (int): 需要爬取的笔记数量

    Returns:
        None: 数据存储在传入的列表中
    """
    response = Selector(text=html_content)
    divs = response.xpath('//div[contains(@class, "feeds-container")]/section/div')  # 选中网页中包含笔记信息的部分

    # 遍历divs获取每一篇笔记的信息
    for div in divs:
        if len(URL_list) >= num:
            break
        
        if div.xpath('.//span[contains(text(), "大家都在搜")]'):
            continue

        # 选择并提取网页数据
        try:
            author_name = div.xpath('.//a[contains(@class, "author")]/span[contains(@class, "name")]/text()').get()  # 作者名字
            like_nr = div.xpath('.//span[contains(@class, "count")]/text()').get()  # 获赞数量
            url = div.xpath('.//a[contains(@class, "cover")]/@href').get()  # 笔记URL
            user_url = div.xpath('.//a[contains(@class, "author")]/@href').get()  # 用户URL
            
            authorName_list.append(author_name)
            likeNr_list.append(like_nr)
            URL_list.append(url)
            userURL_list.append(user_url)

            time.sleep(0.35)
            
        except Exception as e:
            print(f"解析笔记时出错: {e}")
            pass
    
    return True

authorName_list, likeNr_list, URL_list, userURL_list = [], [], [], []
qbar = tqdm(total=num, desc="已获取的笔记数量...")

# 检查是否已经爬取足够数量的笔记，或是否已经达到页面底部
while len(URL_list) < num:
    if '- THE END -' in browser.page_source:
        print(f"当前与{key_word}有关的笔记数量少于 {num}")
        print('检查时间:', time.ctime())
        break
    
    parsePage(browser.page_source, authorName_list, likeNr_list, URL_list, userURL_list, num)
    qbar.update(1)

    if len(URL_list) < num:
        browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')  # 模拟鼠标滚动
        time.sleep(random.uniform(3, 5))

if len(URL_list) > num:
    URL_list = URL_list[:num]
    authorName_list = authorName_list[:num]
    likeNr_list = likeNr_list[:num]
    userURL_list = userURL_list[:num]

qbar.close()

if all(len(lst) == num for lst in [authorName_list, likeNr_list, URL_list, userURL_list]):
    print(f"所有属性列表长度均为 {num}，爬取成功！")
    print(f'检查时间:', time.ctime())
else:
    min_length = min(map(len, [authorName_list, likeNr_list, URL_list, userURL_list]))
    print(f"当前属性列表长度最小值为 {min_length}，请重新运行上一代码单元，直至所有属性列表长度均为 {num}！")
    print(f'检查时间:', time.ctime())

# 清洗数据
likeNr_list = extract_large_number(likeNr_list)
URL_list = [re.sub(r'^/search_result/', '/', url) if url is not None else '' for url in URL_list]
userURL_list = [url.split('/')[-1] if url is not None else '' for url in URL_list]

print("以下为清洁数据示例:\n")
for i in range(min(3, len(authorName_list))):
    print("author_name:", authorName_list[i])
    print("like_nr:", likeNr_list[i])
    print("url:", URL_list[i])
    print("user_url:", userURL_list[i])
    print("------")

def parse_note_page(browser, url, commentNr_list, content_list, datePublished_list, images_list, starNr_list):
    """
    解析单个笔记页面并提取所需数据
    
    Args:
        browser: Selenium WebDriver 实例，用于获取页面内容
        url: 笔记的URL
        commentNr_list (list): 存储评论数量的列表
        content_list (list): 存储笔记内容的列表
        datePublished_list (list): 存储发布时间的列表
        images_list (list of list): 存储图片链接的嵌套列表
        starNr_list (list): 存储收藏数量的列表

    Returns:
        None: 将提取的数据添加到相应的列表中
    """
    whole_url = 'https://www.xiaohongshu.com/explore' + url  # 构造完整笔记URL
    browser.get(whole_url)
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@name="description"]')))  # 等待页面加载完成
    html = browser.page_source
    selector = Selector(text=html)
    
    try:
        # 选择并提取网页数据
        comment_nr = selector.xpath('//*[@class="total"]/text()').get()  # 评论数量
        content = selector.xpath('//*[@name="description"]/@content').get()  # 内容
        datePublished = selector.xpath('//*[@class="date"]/text()').get()  # 发布时间
        
        # 提取图像链接（主图和头像）
        main_image = selector.xpath('.//a[contains(@class, "cover")]/img/@src').get()
        avatar_image = selector.xpath('.//a[contains(@class, "author")]/img/@src').get()
        images = [main_image, avatar_image] if main_image and avatar_image else []
        
        star_nr = selector.xpath('//*[@class="count"]/text()').get()  # 收藏数量

        commentNr_list.append(comment_nr)
        content_list.append(content)
        datePublished_list.append(datePublished)
        images_list.append(images)
        starNr_list.append(star_nr)

    except Exception as e:
        print(f"解析笔记页面时出错: {e}")
        commentNr_list.append(None)
        content_list.append(None)
        datePublished_list.append(None)
        images_list.append([])
        starNr_list.append(None)
        
commentNr_list, content_list, datePublished_list, images_list, starNr_list = [], [], [], [], []

qbar = tqdm(total=len(URL_list), desc="已获取的笔记数量...")
for url in URL_list:
    parse_note_page(browser, url, commentNr_list, content_list, datePublished_list, images_list, starNr_list)
    qbar.update(1)
    time.sleep(random.uniform(0.5, 2))

qbar.close()

if all(len(lst) == num for lst in [commentNr_list, content_list, datePublished_list, images_list, starNr_list]):
    print(f"所有属性列表长度均为 {num}，爬取成功！")
    print(f'检查时间:', time.ctime())
else:
    min_length = min(map(len, [commentNr_list, content_list, datePublished_list, images_list, starNr_list]))
    print(f"当前属性列表长度最小值为 {min_length}，请重新运行上一代码单元，直至所有属性列表长度均为 {num}！")
    print(f'检查时间:', time.ctime())

# 清洗更多数据
commentNr_list = extract_large_number(extract_number(commentNr_list))
starNr_list = extract_large_number(starNr_list)
datePublished_list = extract_date(datePublished_list)

print("以下为清洁数据示例:\n")
for i in range(min(3, len(commentNr_list))):
    print("comment_nr:", commentNr_list[i])
    print("content:", content_list[i])
    print("datePublished:", datePublished_list[i])
    print("images:", images_list[i])
    print("star_nr:", starNr_list[i])
    print("------")

# 函数：拦截并下载图像
def download_images_with_selenium_wire(browser, images_list, folder_path='images'):
    """
    使用 Selenium Wire 拦截并下载图像。

    Args:
        browser: Selenium Wire WebDriver 实例。
        images_list (list of list): 包含图像URL的嵌套列表。
        folder_path (str): 图像保存的目标文件夹路径。

    Returns:
        None
    """
    # 将所有图像URL扁平化为一个列表
    flat_images = [img_url for sublist in images_list for img_url in sublist if img_url]

    # 使用集合去重，避免重复下载同一张图像
    unique_images = list(set(flat_images))

    print(f"共找到 {len(unique_images)} 张唯一图像需要下载。")

    for img_url in tqdm(unique_images, desc="下载图像"):
        try:
            # 清除之前的请求记录
            browser.scopes = ['.*']  # 只拦截所有请求
            browser.requests.clear()

            # 导航到图像URL
            browser.get(img_url)

            # 等待图像响应
            for request in browser.requests:
                if request.response and request.url == img_url and 'image' in request.response.headers.get('Content-Type', ''):
                    image_data = request.response.body

                    # 确定图片的扩展名
                    img_extension = os.path.splitext(img_url.split('?')[0])[1]
                    if img_extension.lower() not in ['.jpg', '.jpeg', '.png', '.gif', '.bmp']:
                        img_extension = '.jpg'  # 默认扩展名

                    # 构造图片文件名，避免重复
                    img_filename = f"{hash(img_url)}{img_extension}"
                    img_path = os.path.join(folder_path, img_filename)

                    # 保存图片到本地
                    with open(img_path, 'wb') as f:
                        f.write(image_data)
                    break
        except Exception as e:
            print(f"下载图像 {img_url} 时出错：{e}")

# 下载图像
download_images_with_selenium_wire(browser, images_list, images_folder)

# 构建数据字典
dic = {
    # "author_collect_nr": authorCollectNr_list,# 作者获赞与收藏数量
    # "author_fans_nr": authorFansNr_list,# 粉丝数量
    "author_name": authorName_list,  # 作者名字
    # "author_note_nr": authorNoteNr_list,# 作者笔记数量
    "comment_nr": commentNr_list,  # 笔记评论数量
    "content": content_list,  # 笔记内容
    "datePublished": datePublished_list,  # 笔记发布日期
    "images": images_list,  # 笔记封面图片（嵌套列表）
    "like_nr": likeNr_list,  # 笔记获赞数量
    "star_nr": starNr_list,
    "url": URL_list,  # 笔记URL
    "user_url": userURL_list  # 作者URL
}

df = pd.DataFrame.from_dict(dic)
df = df.explode('images')  # 展开嵌套的图像列表
df = df[~df.duplicated(keep='first')]  # 检索并删除所有属性值都相同的行,即保留第一次出现的行，删除后续的重复行
print("删除", num - len(df), "行重复行后剩余", len(df), "行。")
print('检查时间:', time.ctime())
display(df.head(10))
df.to_csv('scraped_data.csv', encoding='utf-8-sig')
print("数据已保存到 'scraped_data.csv'")



即将开始检查小红书登录状态...
爬取数据有账户封禁的风险，建议使用非主账号登录。
登录成功
检查时间: Wed Dec  4 19:19:21 2024
请在文本框中根据提示输入搜索关键词和笔记爬取数量。
即将开始检查网页加载状态...
如果网页进入人机验证页面，请先手动完成验证。
加载成功
检查时间: Wed Dec  4 19:19:45 2024
已自动更改模式为图文。
请选择排序方式:
1. 综合
2. 最新
3. 最热
请输入有效的排序方式...


已获取的笔记数量...:  20%|██        | 1/5 [00:01<00:07,  1.79s/it]


所有属性列表长度均为 5，爬取成功！
检查时间: Wed Dec  4 19:19:48 2024
以下为清洁数据示例:

author_name: 易度泽泽-留学咨询
like_nr: 1
url: /66e556d9000000001e01a021?xsec_token=ABUngLbv791gP_tYthlYIVXSJB1Z1VMEPsXCdgCzrVtsI=&xsec_source=
user_url: 66e556d9000000001e01a021?xsec_token=ABUngLbv791gP_tYthlYIVXSJB1Z1VMEPsXCdgCzrVtsI=&xsec_source=
------
author_name: NYUSH-NYUStern商科硕士
like_nr: 27
url: /6748403d000000000202cd3a?xsec_token=ABm6Qf522fzI2bYexD9bSFL-Eqt321sTe_PxIhosUXiCI=&xsec_source=
user_url: 6748403d000000000202cd3a?xsec_token=ABm6Qf522fzI2bYexD9bSFL-Eqt321sTe_PxIhosUXiCI=&xsec_source=
------
author_name: 小红薯666C2C5D
like_nr: 1
url: /66920ed4000000002500032a?xsec_token=AB--gWjR_PTnn6MI8rkQBRlwYgv4bsk6rA7odpHQVcdJY=&xsec_source=
user_url: 66920ed4000000002500032a?xsec_token=AB--gWjR_PTnn6MI8rkQBRlwYgv4bsk6rA7odpHQVcdJY=&xsec_source=
------


已获取的笔记数量...: 100%|██████████| 5/5 [00:24<00:00,  4.93s/it]


所有属性列表长度均为 5，爬取成功！
检查时间: Wed Dec  4 19:20:13 2024
以下为清洁数据示例:

comment_nr: None
content: 25fall的上纽申请已经开放啦！想要去上纽商学院的同学们材料准备要到尾声了，不然就很紧急了！🔥🔥🔥 今年度度依旧给大家分享了商学院四大专业的硕士班级情况！ 	 👉🏻👉🏻易度分析相比与2024届 25届班级情况的差距 作为连续3年都有上纽商科offer的申请团队，我们也做了数据的整合， 1⃣️.没有公布GRE的参考分数，但是实际情况是不少同学还是提交了的GRE的成绩的，并且基本都是320+以上的成绩 2⃣️中国本科的同学占比相比去年占比要多了，基本在30-40%，而去年美本或者海本的数量接近快90%的占比，大家都觉得真的非常难申。陆本同学的申请也非常的卷。 据我们易度今年申请提交情况，有985/211背景的居多，双非一本财经也有，但是整体录取来看情况一般。在美本背景的录取上，还是非常具有优势 3⃣️均分今年有3.5+的趋势，往年基本的录取都在3.6-3.8之间，美本3.3今年也有offer！ 4⃣️看似申请标准好像松了一些，其实并没有不卷，四个专业的申请都是非常的热门 5⃣️今年的最早申请截止日期是10月15日，所以还没有准备齐全申请材料同学加快进度！ 	 🌟🌟申请材料： 1. 申请材料清单个人信息简历全日制本科（或本科及硕士） 2.成绩单1封推荐信2篇文书（专业短文及个人描述短文） 3.先修课程GMAT或GRE（可选）*GMAT或GRE非必须要求，如选择提交分数，建议您提交线下考试中心成绩 4. TOEFL 100+；IELTS7+ 先修课程-根据各专业有单独的要求，可戳度度给您详细发送找生简章 	 ⚠️⚠️上纽大的4个商科硕士就业表现都很不错，具体可以看2023届毕业生就业报告 所以建议感兴趣的同学尽早提交申请，因为这4个项目开放申请比较早，结束申请也比较早 早申请机会往往更大一些！ 	 如果你也对商科硕士项目感兴趣，拿到名校offer ✅加入易度“菁英计划”“启明星计划”“种子计划”！ 🌟 易度也匹配了生物大方向来自康奈尔的申请导师为大家服务文书和规划 💯100+外籍文书导师文书修正，全海归毕业团队打造申请 📲全天在线接受宝子们的线上线下留学咨询 	 #留学申请季  #留学美国  #美国

下载图片: 100%|██████████| 5/5 [00:00<00:00, 156503.88it/s]

删除 0 行重复行后剩余 5 行。
检查时间: Wed Dec  4 19:20:13 2024


,author_name,comment_nr,content,datePublished,images,like_nr,star_nr,url,user_url
0,易度泽泽-留学咨询,NaN,25fall的上纽申请已经开放啦！想要去上纽商学院的同学们材料准备要到尾声了，不然就很紧急了...,2024-09-14,NaN,1,1,/66e556d9000000001e01a021?xsec_token=ABUngLbv7...,66e556d9000000001e01a021?xsec_token=ABUngLbv79...
1,NYUSH-NYUStern商科硕士,NaN,期末将至，NYU Shanghai - NYU Stern商科硕士项目的学生大使们精心推荐了...,None,NaN,27,1,/6748403d000000000202cd3a?xsec_token=ABm6Qf522...,6748403d000000000202cd3a?xsec_token=ABm6Qf522f...
2,小红薯666C2C5D,NaN,上纽4个热门商科硕士，2025年入学招生时间表公布！首轮申请10月截止！ 各专业都安排三个...,2024-07-13,NaN,1,1,/66920ed4000000002500032a?xsec_token=AB--gWjR_...,66920ed4000000002500032a?xsec_token=AB--gWjR_P...
3,留学客栈,NaN,NYU Stern商学院 (上海纽大）offer 3️⃣周下 一年毕业🎓非常快，在上海上课...,2024-10-20,NaN,2,1,/6715c342000000001b03d845?xsec_token=ABB8nFeSq...,6715c342000000001b03d845?xsec_token=ABB8nFeSqW...
4,阿鱼留学咨询,11.0,上海纽约大学24fall商科硕士申请通道已经开放 💜上海纽约大学和纽约大学斯特恩（Stern...,2023-08-09,NaN,176,1,/64d39c4d000000001201fd05?xsec_token=AB8PWO8-l...,64d39c4d000000001201fd05?xsec_token=AB8PWO8-lB...


数据已保存到 'scraped_data.csv'
